In [3]:
!ls

'3D structures.sdf'   convertFASTAtoSDF.py
 ConvertSDF.ipynb    'drug sequences.fasta'
 README.txt	      extract.py
 __pycache__	      pharmacologically_active_extracted.csv
 all_extracted.csv    protein.fasta


In [4]:
import sys
import os
import pandas as pd
import shutil

In [37]:
def getAllSDFname(fileTXT):
    allName = []
    with open(fileTXT, 'r+', encoding="utf-8") as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            line = lines[i]
            # print (line)
            if line.startswith("> <DATABASE_ID"):
                ne = lines[i + 1][:-1]
                allName.append(str(ne))

    ret_df = pd.DataFrame({"All": allName})
    ret_file_path = fileTXT + ""
    ret_file_path =  "extract_" + ret_file_path.replace("-copy.txt", ".csv")
    ret_df.to_csv(ret_file_path, index=False)
    print(len(allName))
    return allName


def split_3D(allName, location_path, fileTXT):
    indexFile = 0
    close = 1

    file1 = open(fileTXT, 'r', encoding="utf-8")
    try:
        while indexFile < len(allName) + 1:
            line = file1.readline()
            if close == 1:
                if indexFile >= len(allName):
                    break
                # Write file .txt
                file2 = open(
                    location_path + allName[indexFile] + ".txt", "w", encoding="utf-8")

                indexFile = indexFile + 1
                close = 0
            # if line.startswith("DB"): print(line)
            if not line.startswith("$$$$"):
                file2.write(line)
            else:
                file2.write("$$$$")
                file2.close()
                close = 1

                # Convert to sdf file
                thisFile = location_path + allName[indexFile - 1] + ".txt"
                base = os.path.splitext(thisFile)[0]
                os.rename(thisFile, base + ".sdf")

            # if line is empty
            # end of file is reached
            if not line:
                break
    finally:
        file1.close()

In [39]:
def getAllFASTAname(fileTXT):
    allName = []
    with open(fileTXT, 'r+', encoding="utf-8") as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            line = lines[i]
            if line.startswith(">drugbank"):
                # if '|' in lines[i][15:-1]: print(lines[i][15:22] + '*')
                name = lines[i].split()
                name = name[0].split("|")
                name = name[1]
                allName.append(str(name))

    ret_df = pd.DataFrame({"All": allName})
    ret_file_path = fileTXT + ""
    ret_file_path = "extract_" + ret_file_path.replace("-copy.txt", ".csv")
    ret_df.to_csv(ret_file_path, index=False)
    print(len(allName))
    return allName


def split_FASTA(allName, location_path, fileTXT):
    indexFile = -1
    resultDict = {}
    buff = ""
    file1 = open(fileTXT, 'r', encoding="utf-8")
    try:
        while 1:
            line = file1.readline()
            if line.startswith(">drugbank"):
                if(indexFile != -1):
                    if allName[indexFile] not in resultDict:
                        resultDict[allName[indexFile]] = buff
                    else:
                        if len(buff) > len(resultDict[allName[indexFile]]):
                            resultDict[allName[indexFile]] = buff
                    
                buff = line
                indexFile = indexFile + 1
            else:
                buff = buff + line
            # if line is empty
            # end of file is reached
            if not line:
                buff = buff + '\n' # for final file 
                if allName[indexFile] not in resultDict:
                    resultDict[allName[indexFile]] = buff
                else:
                    if len(buff) > len(resultDict[allName[indexFile]]):
                        resultDict[allName[indexFile]] = buff
                break
    finally:
        file1.close()
    print("Not duplicate: ", len(resultDict))
    for key in resultDict:
        file2 = open(location_path + key + ".txt", "w", encoding="utf-8")
        file2.write(resultDict[key][:-1])
        file2.close()
        
        # Convert to sdf file
        thisFile = location_path + key + ".txt"
        base = os.path.splitext(thisFile)[0]
        os.rename(thisFile, base + ".fasta")


def removeAllInFolder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [35]:
def extract(folder_name, file_name):
    # Convert the copy file_name from .... to .txt
    base, ext = os.path.splitext(file_name)
    thisFile = base + "-copy" + ext
    shutil.copy2(file_name, thisFile)
    fileTXT = base + "-copy" + ".txt"
    os.rename(thisFile, fileTXT)
    
    # Create new folder to save result
    newpath = folder_name 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    location_path = './'+ folder_name +'/'
    try:
        if ext == '.sdf':
            allName = getAllSDFname(fileTXT)
            # Read big file and split to small file and put in folder  split3D
            
            removeAllInFolder(location_path)
            split_3D(allName, location_path, fileTXT)
        elif ext == '.fasta':
            allName = getAllFASTAname(fileTXT)
            # Read big file and split to small file and put in folder  splitFASTA
            removeAllInFolder(location_path)
            split_FASTA(allName, location_path, fileTXT)

    finally:
        os.remove(fileTXT)


# python extract.py "3D structures_source.sdf"
# python extract.py "drug sequences.fasta"


In [40]:
# extract("3D structures_sdf", "3D structures.sdf")
extract("drug sequences_fasta", "drug sequences.fasta")
extract("protein_fasta", "protein.fasta")

360
Not duplicate:  233
5260
Not duplicate:  4867


In [ ]:
### Convert Fasta to SDF

In [43]:
# import openbabel
from openbabel import openbabel
import pandas as pd
from extract import removeAllInFolder


def getColumnData(file_name, column_name):
    fields = [column_name]
    df = pd.read_csv(file_name, skipinitialspace=True, usecols=fields)
    data = (list(df[column_name]))
    data = list(dict.fromkeys(data))  # remove duplicate
    return data


def convertFASTAtoSDF(list_file, folder_name):
    fromFolder = './'+folder_name+'/'
    resultFolder = './Convert_'+folder_name+'/'
    
    # Create new folder to save result
    newpath = resultFolder 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    removeAllInFolder(resultFolder)
    for x in list_file:
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("fasta", "sdf")

        obConversion.OpenInAndOutFiles(
            fromFolder + x + '.fasta', resultFolder + x + '.sdf')
        obConversion.Convert()
        obConversion.CloseOutFile()

In [44]:
allDrugs = getColumnData('all_extracted.csv', 'Drug IDs')
allSDF = getColumnData('extract_3D structures.csv', 'All')
allFASTA = getColumnData('extract_drug sequences.csv', 'All')

drugNotInSDF = []
for drug in allDrugs:
    if drug not in allSDF:
        drugNotInSDF = drugNotInSDF + [drug]

print("Drug not in SDF: ", len(drugNotInSDF))

drugInFasta = []
for drug in drugNotInSDF:
    if drug in allFASTA:
        drugInFasta = drugInFasta + [drug]

print("Drug not in SDF but In Fasta: ", len(drugInFasta))

print("Drug not in SDF and not in FASTA: ",
      len(drugNotInSDF) - len(drugInFasta))

convertFASTAtoSDF(drugInFasta, "drug sequences_fasta")


# conda install -c conda-forge openbabel


Drug not in SDF:  1644
Drug not in SDF but In Fasta:  187
Drug not in SDF and not in FASTA:  1457


In [45]:
allProt = getColumnData('all_extracted.csv', 'UniProt ID')
allProtFasta = getColumnData('extract_protein.csv', 'All')

ProtInFasta = []
for prot in allProt:
    if prot in allProtFasta:
        ProtInFasta = ProtInFasta + [prot]

print("Prot have in protein_fasta: ", len(ProtInFasta))
print("Prot not have in protein_fasta: ", len(allProt)-len(ProtInFasta))


convertFASTAtoSDF(ProtInFasta, "protein_fasta")

Prot have in protein_fasta:  4867
Prot not have in protein_fasta:  0


In [48]:
# Protein is in Convert_protein_fasta
# Ligand is in 3D structures_sdf + Convert_drug sequences_fasta

# all_active_pair                 in pharmacologically_active_extracted
# all active and in active = in all_extracted.csv


from shutil import copyfile

def load_uniprotid_drugid(file_path, prot_col, drug_col):
    df = pd.read_csv(file_path)
    protids = df[prot_col].values
    drugids = df[drug_col].values
    ret = [(protids[i],drugids[i]) for i in range(len(protids))]
#     print(ret)
    return ret


all_active_pair = load_uniprotid_drugid("pharmacologically_active_extracted.csv", "UniProt ID", "Drug IDs")
all_pair = load_uniprotid_drugid("all_extracted.csv", "UniProt ID", "Drug IDs")
all_inactive_pair = []

for pair in all_pair:
    if pair not in all_active_pair:
        all_inactive_pair = all_inactive_pair + [pair]
print(len(all_active_pair))
print(len(all_inactive_pair))
        
allSDF = getColumnData('extract_3D structures.csv', 'All')
allFASTA = getColumnData('extract_drug sequences.csv', 'All')

for x in all_active_pair[:1]:
    prot = x[0]
    drug = x[1]
    if drug not in allSDF and drug not in allFASTA:
        print(drug)
        continue

    nameFolder = prot + "_CHEMBL" + drug
    
    # Create new folder to save result
    newpath = "dude1/" + nameFolder 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
#     try:
    src = './Convert_protein_fasta/' + prot + ".sdf"
    dst = newpath + "/" + prot + ".sdf"
    copyfile(src, dst)

    src = None
    if prot in allSDF:
        src = './3D structures_sdf/' + drug + ".sdf"
    else:
        src = './Convert_drug sequences_fasta/' + drug + ".sdf"
    dst = dst = newpath + "/" + drug + ".sdf"
    copyfile(src, dst)


5842
15523


FileNotFoundError: [Errno 2] No such file or directory: './Convert_drug sequences_fasta/DB00303.sdf'